In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import json
import pandas as pd
import time
import random
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

import datetime
def date_time():return datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=5, minutes=30))).strftime("%Y-%m-%d_%H-%M-%S")

- per page count show in website 15 but we can get 150 per page
-


- Inner page code

    params = {
        'backurl': 'H4sIAAAAAAAAAEXLMQ7CMAyF4dtkLQyMFmKBgQ5I9AJWYiBSEwfbKcrtKSKI7X+f9IaCd9LBcyqY41pKKP6xfwI6yhatTa2QwmEc+74aWtWTcC1fjtmzlKNwgt4Tg8MQhFT/7w5nai+WAE4NxWDj5piiwXbn9LZi+DwXzJ7CBTPNYFLJJQ4EP38De7CP8bMAAAA=',
    }

    response = requests.get(
        'https://app.companiesoffice.govt.nz/companies/app/ui/pages/companies/979583',
        params=params,
        headers=headers,
    )


In [ ]:
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive',
    'Referer': 'https://app.companiesoffice.govt.nz/companies/app/ui/pages/companies/search?mode=standard&type=entities&q=a&advancedPanel=true&entityTypes=ALL&entityStatusGroups=ALL',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Google Chrome";v="137", "Chromium";v="137", "Not/A)Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    # 'Cookie': 'visid_incap_1231483=9WAgHGwBTbWU/4hsw8W1wZjSQmgAAAAAQUIPAAAAAAAO+mEmJrPgAh0hc+mvQjEg; _ga=GA1.1.1800181697.1749209755; _gcl_au=1.1.388435986.1749209755; _hjSessionUser_1332618=eyJpZCI6IjRjNTMzM2NlLWVjODgtNWU1NS05MGNlLTViZGEwMGU1NmI1ZSIsImNyZWF0ZWQiOjE3NDkyMDk3NTUzNTEsImV4aXN0aW5nIjp0cnVlfQ==; nlbi_1231483=B2IrebtA3Hl+D02xtbLFkAAAAAB4kEQ4qUEx4Ljkf8kcQmIL; mf5=-625347562; incap_ses_1565_1231483=gEVVHRbArEj5ljwkif+3FbbWS2gAAAAAVT9bwkJLbkBbKMmlhDLJzw==; _hjSession_1332618=eyJpZCI6IjA2MmVkZjVhLWJkYWQtNGI5NS04YjllLTIwYmJmZTcyMGJmMSIsImMiOjE3NDk4MDA2MzA3NTEsInMiOjAsInIiOjAsInNiIjowLCJzciI6MCwic2UiOjAsImZzIjowLCJzcCI6MX0=; _ga_4T47RXVMFX=GS2.1.s1749799964$o7$g1$t1749801446$j60$l0$h0; _ga_KRN17JQ9HG=GS2.1.s1749799964$o7$g1$t1749801446$j60$l0$h0',
}

params = {
    'q': 'a',
    'entityTypes': 'ALL',
    'entityStatusGroups': 'ALL',
    'incorpFrom': '',
    'incorpTo': '',
    'addressTypes': 'ALL',
    'addressKeyword': '',
    'start': '1',
    'limit': '1000',
    'sf': '',
    'sd': '',
    'advancedPanel': 'true',
    'mode': 'advanced',
}

# response = requests.get(
#     'https://app.companiesoffice.govt.nz/companies/app/ui/pages/companies/search',
#     params=params,
#     headers=headers,
# )

In [ ]:
# response

In [ ]:
# soup = BeautifulSoup(response.text, 'html.parser')

### Companies List

In [ ]:
def inner_exract(tag):
    result = {}
    if tag.find('a', class_='link') is not None:

        a_tag = tag.find('a', class_='link')

        href = a_tag.get('href', '')

        result['href'] = href

        match = re.search(r"viewCompanyDetails\('(\d+)'\)", href)
        if match:
            result['company_id'] = match.group(1)

        nzbn = re.search(r'NZBN:\s*(\d+)', tag.text)

        if nzbn:
            result['nzbn'] = nzbn.group(1)

        name_tag = a_tag.find('span', class_='entityName')
        if name_tag:
            result['company_name'] = name_tag.get_text(strip=True)

        return result
    else:
        return {}

In [ ]:
from bs4 import BeautifulSoup
import re


def parse_company_html(res, letter, page) -> dict:
    soup = BeautifulSoup(res.text, 'html.parser')
    tags = soup.find_all('td', class_='removed')

    print(len(tags))

    with ThreadPoolExecutor(max_workers=4) as exe:
        results = [i for i in tqdm(exe.map(inner_exract, tags), total=len(tags))]

    # collect_companies_list = [j for i in results for j in i]

    return results

In [ ]:
# soup.find_all('td', class_='removed')[310]
# parse_company_html(soup.find_all('td', class_='removed')[300])

In [1]:
def collect_companies_res(letter, page):
    params = {
    'q': letter,
    'entityTypes': 'ALL',
    'entityStatusGroups': 'ALL',
    'incorpFrom': '',
    'incorpTo': '',
    'addressTypes': 'ALL',
    'addressKeyword': '',
    'start': page,
    'limit': '1000',
    'sf': '',
    'sd': '',
    'advancedPanel': 'true',
    'mode': 'advanced',
    }

    response = requests.get(
    'https://app.companiesoffice.govt.nz/companies/app/ui/pages/companies/search',
    params=params,
    headers=headers)

    return letter, response, page

In [ ]:
# soup.find('div', class_='totalInfo').find('h4').text.split('results')[0].split('of')[1].strip()

In [ ]:
import string

In [ ]:
collected_all_companies = []
for letter in tqdm(string.ascii_lowercase[15:19]):
    params = { 'q': letter, 'entityTypes': 'ALL', 'entityStatusGroups': 'ALL', 'incorpFrom': '', 'incorpTo': '', 'addressTypes': 'ALL', 'addressKeyword': '', 'start': 0, 'limit': '10', 'sf': '', 'sd': '', 'advancedPanel': 'true', 'mode': 'advanced', }
    response = requests.get( 'https://app.companiesoffice.govt.nz/companies/app/ui/pages/companies/search', params=params, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    tags = soup.find_all('td', class_='removed')

    h4_tag = soup.find('div', class_='totalInfo')

    text = h4_tag.find('h4').get_text(strip=True)
    # Example input: "Showing 1 to 20 of 1063 results"
    try:

      page_ob = re.search(r'of\s+(\d+)\s+results', text).group(1)

      pages = int(int(page_ob)/1000)+1
      # pages = int(10)

      print((letter, 0, page_ob, pages))

      with ThreadPoolExecutor(max_workers=4) as exe:
           results = [i for i in tqdm(exe.map(collect_companies_res, [letter]*pages ,range(0, pages)), total=pages)]

      collected_all_companies.extend(results)

    except:
      pass



  0%|          | 0/4 [00:00<?, ?it/s]

('p', 0, '117081', 118)



 25%|██▌       | 1/4 [00:53<02:40, 53.55s/it]

('q', 0, '10071', 11)



 50%|█████     | 2/4 [01:26<01:23, 41.68s/it]

('r', 0, '97847', 98)



 75%|███████▌  | 3/4 [05:00<02:00, 120.38s/it]

('s', 0, '174190', 175)



100%|██████████| 4/4 [11:13<00:00, 168.39s/it]


In [ ]:
collected_all_companies[0]

('q', <Response [200]>, 0)

In [ ]:
total_companies = [parse_company_html(i[1], i[0], i[2]) for i in tqdm(collected_all_companies)]

  0%|          | 0/284 [00:00<?, ?it/s]

1138



  0%|          | 1/284 [00:01<08:45,  1.86s/it]

1136



  1%|          | 2/284 [00:04<10:17,  2.19s/it]

...


 99%|█████████▉| 282/284 [08:29<00:03,  1.54s/it]

936



100%|█████████▉| 283/284 [08:31<00:01,  1.53s/it]

934



100%|██████████| 284/284 [08:32<00:00,  1.80s/it]


In [ ]:
total_companies[0]

[{'href': "javascript:viewCompanyDetails('3413313');",
  'company_id': '3413313',
  'nzbn': '9429031068376',
  'company_name': 'Q N Q LIMITED'},
 {},
 {'href': "javascript:viewCompanyDetails('1101508');",
  'company_id': '1101508',
  'nzbn': '9429037079093',
  'company_name': 'Q & Q RESEARCH LIMITED'},
 {},
 {'href': "javascript:viewCompanyDetails('3492818');",
  'company_id': '3492818',
  'nzbn': '9429031001564',
  'company_name': 'Q VANTI LIMITED'},
 {},
 {'href': "javascript:viewCompanyDetails('1875537');",
  'company_id': '1875537',
  'nzbn': '9429033812229',
  'company_name': 'Q FUND LIMITED'},
 {},
 {'href': "javascript:viewCompanyDetails('1897900');",
  'company_id': '1897900',
  'nzbn': '9429033674100',
  'company_name': 'Q NZ HOLDINGS LIMITED'},
 {},
 {'href': "javascript:viewCompanyDetails('1555224');",
  'company_id': '1555224',
  'nzbn': '9429035188674',
  'company_name': 'Q WEAR LIMITED'},
 {},
 {'href': "javascript:viewCompanyDetails('2194186');",
  'company_id': '2194186

In [ ]:
total_companies_cv = [item for sublist in total_companies for item in sublist if len(item)>0]

In [ ]:
output_df = pd.DataFrame(total_companies_cv)

In [ ]:
output_df

,href,company_id,nzbn,company_name
0,javascript:viewCompanyDetails('3413313');,3413313,9429031068376,Q N Q LIMITED
1,javascript:viewCompanyDetails('1101508');,1101508,9429037079093,Q & Q RESEARCH LIMITED
2,javascript:viewCompanyDetails('3492818');,3492818,9429031001564,Q VANTI LIMITED
3,javascript:viewCompanyDetails('1875537');,1875537,9429033812229,Q FUND LIMITED
4,javascript:viewCompanyDetails('1897900');,1897900,9429033674100,Q NZ HOLDINGS LIMITED
...,...,...,...,...
153843,javascript:viewCompanyDetails('5572006');,5572006,9429041579053,S J CABINETRY LIMITED
153844,javascript:viewCompanyDetails('4468398');,4468398,9429030196827,S J CARS LIMITED
153845,javascript:viewCompanyDetails('8010296');,8010296,9429048250184,S J CATERS LIMITED
153846,javascript:viewCompanyDetails('2110206');,2110206,9429032841466,S J CHOPRA LIMITED


In [ ]:
string.ascii_lowercase[0:5]

'abcde'

In [ ]:
output_df.to_csv(f'/content/drive/MyDrive/Company Project/NZ_Company_Data_From_CompaniesOfficeGovtNz/data/companies_list_from_use_keywords_klmn_output_file_{date_time()}.csv')

In [ ]:
15-26

-11

### Inner Pgae Details

In [ ]:
cmpdf = pd.read_csv('/content/drive/MyDrive/Company Project/NZ_Company_Data_From_CompaniesOfficeGovtNz/data/nz_company_registry_scraper_companiesofficegovtnz_companies_list_16-06-2025.csv')

In [ ]:
cmpdf.columns

Index(['Unnamed: 0', 'href', 'company_id', 'nzbn', 'company_name'], dtype='object')

In [ ]:
cmpdf.head(1)

,Unnamed: 0,href,company_id,nzbn,company_name
0,0,javascript:viewCompanyDetails('174864');,174864,9429040149837,F F F ETCHELLS LIMITED


In [ ]:
def inner_page_res(num):
    res = requests.get(f'https://app.companiesoffice.govt.nz/companies/app/ui/pages/companies/{num}', headers=headers)
    return res, num

In [ ]:
result = inner_page_res(1170995)

In [ ]:
result

(<Response [200]>, 1170995)

In [ ]:
def clean_text(el):
    return el.get_text(strip=True).replace('\xa0', ' ')

In [ ]:
def clean_special(text):
    return re.sub(r'''[^a-zA-Z0-9\s,.'"]''', '', text)

In [ ]:
def inner_page_extract(res, num):
    data = {}
    address = {}
    soup = BeautifulSoup(res.text, 'html.parser')
    for litag in soup.find('div', class_='readonly companySummary').find_all('div', class_='row'):
        try:

          if any(tag in str(litag).lower() for tag in ['registered', 'address']):
                address[clean_text(litag.strong)] = clean_text(litag.br)
          else:
                table_text = clean_text(litag.label)
                data[table_text] = clean_special(clean_text(litag).split(table_text)[1])
        except:pass

    data['address'] = address
    data['company_id'] = num

    return data

In [ ]:
inner_page_extract(result[0], result[1])

{'Company number:': '1170995',
 'NZBN:': '9429036723638',
 'Incorporation Date:': '12 Nov 2001',
 'Entity type:': 'NZ Limited Company',
 'Constitution filed:': 'No',
 'AR filing month:': 'May\r\n              \r\n              , last filed on29 May 2025',
 'Ultimate holding company': 'NoEdit',
 'Directors': 'Showing 1 of 1 directors',
 'Company record link:': 'httpsapp.companiesoffice.govt.nzco1170995',
 'address': {'Registered office address': '',
  'Registered Office': 'OFFICE SUITE ONE, 3 Mckenzie Street, Levin, 5510\r\n, New Zealand',
  'Address for service': 'OFFICE SUITE ONE, 3 Mckenzie Street, Levin, 5510\r\n, New Zealand'},
 'company_id': 1170995}

In [ ]:
with ThreadPoolExecutor(max_workers=2) as exe:
    results = [i for i in tqdm(exe.map(inner_page_res, cmpdf['company_id']), total=len(cmpdf['company_id']), desc='Inner Page')]

Inner Page:   0%|          | 18/13902 [01:57<25:06:22,  6.51s/it]


KeyboardInterrupt: 

In [ ]:
with ThreadPoolExecutor(max_workers=4) as exe:
    results = [i for i in tqdm(exe.map(inner_page_extract, [i[0] for i in results], [i[1] for i in results]), total=len(results))]